# Assigment 4

In [1]:
import math
import numpy as np
import scipy.sparse as sp

import igl
import meshplot as mp

from math import sqrt

In [2]:
v, f = igl.read_triangle_mesh("data/irr4-cyl2.off")
tt, _ = igl.triangle_triangle_adjacency(f)

c = np.loadtxt("data/irr4-cyl2.constraints")
cf = c[:, 0].astype(np.int64)
c = c[:, 1:]

In [3]:
def align_field(V, F, TT, soft_id, soft_value, llambda):
    assert(soft_id[0] > 0)
    assert(soft_id.shape[0] == soft_value.shape[0])

    
    # Edges
    e1 = V[F[:, 1], :] - V[F[:, 0], :]
    e2 = V[F[:, 2], :] - V[F[:, 0], :]

    # Compute the local reference systems for each face, T1, T2
    T1 = e1 / np.linalg.norm(e1, axis=1)[:,None]
        
    T2 =  np.cross(T1, np.cross(T1, e2))
    T2 /= np.linalg.norm(T2, axis=1)[:,None]
  
    # Arrays for the entries of the matrix   For L &
    data = []
    ii = []
    jj = []
    not_cons = []
    
    index = 0
    for f in range(F.shape[0]):
        for ei in range(3): # Loop over the edges
            
            # Look up the opposite face
            g = TT[f, ei]
            
            # If it is a boundary edge, it does not contribute to the energy
            # or avoid to count every edge twice f>g
            if g == -1 or f > g:
                continue
                
            # Compute the complex representation of the common edge
            e  = V[F[f, (ei+1)%3], :] - V[F[f, ei], :]
            
            vef = np.array([np.dot(e, T1[f, :]), np.dot(e, T2[f, :])])
            vef /= np.linalg.norm(vef)
            ef = (vef[0] + vef[1]*1j).conjugate()
            
            veg = np.array([np.dot(e, T1[g, :]), np.dot(e, T2[g, :])])
            veg /= np.linalg.norm(veg)
            eg = (veg[0] + veg[1]*1j).conjugate()
            
            # Add the term conj(f)^n*ui - conj(g)^n*uj to the energy matrix
            # The first one is used to store conjugate ef
            # The second one stores the row where ef is in the L
            # The third one stores the uf
            data.append(ef);  ii.append(index); jj.append(f)
            # similar, the minus because there's minus in dot before
            data.append(-eg); ii.append(index); jj.append(g)

            index += 1
            
    sqrtl = sqrt(llambda)

    
    # Convert the constraints into the complex polynomial coefficients and add them as soft constraints
    # Attention!!! This is for the soft constraints
    # Rhs of the system
    b = np.zeros(index + soft_id.shape[0], dtype=np.complex)
    
    for ci in range(soft_id.shape[0]):
        f = soft_id[ci]
        v = soft_value[ci, :]
        
        # Project on the local frame
        c = np.dot(v, T1[f, :]) + np.dot(v, T2[f, :])*1j
        
        data.append(sqrtl); ii.append(index); jj.append(f)
        b[index] = c * sqrtl
 
        
        index += 1
    
    assert(b.shape[0] == index)
 
    

    
    
    # Solve the linear system
    A = sp.coo_matrix((data, (ii, jj)), shape=(index, F.shape[0])).asformat("csr")
    u = sp.linalg.spsolve(A.H @ A, A.H @ b) #Then we get u
    #in order to visualize u, we need to covert it back 
    R = T1 * u.real[:,None] + T2 * u.imag[:,None]

    return R

In [4]:
def plot_mesh_field(V, F, R, constrain_faces):
    # Highlight in red the constrained faces
    col = np.ones_like(f)
    col[constrain_faces, 1:] = 0
    
    # Scaling of the representative vectors
    avg = igl.avg_edge_length(V, F)/2

    #Plot from face barycenters
    B = igl.barycenter(V, F)

    p = mp.plot(V, F, c=col)
    p.add_lines(B, B + R * avg)

    
    return p

In [5]:
R = align_field(v, f, tt, cf, c, 1e6)
plot_mesh_field(v, f, R, cf)

/var/folders/_m/s6xzvg695mb_m9vh7l4cly740000gn/T/ipykernel_74204/741407221.py:61: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  b = np.zeros(index + soft_id.shape[0], dtype=np.complex)
/Users/yinuochang/miniforge3/envs/gm2022/lib/python3.9/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

# Part1: Hard Constraints

In [6]:
def align_field_hard(V, F, TT, soft_id, soft_value, llambda):
    assert(soft_id[0] > 0)
    assert(soft_id.shape[0] == soft_value.shape[0])

    
    # Edges
    e1 = V[F[:, 1], :] - V[F[:, 0], :]
    e2 = V[F[:, 2], :] - V[F[:, 0], :]

    # Compute the local reference systems for each face, T1, T2
    T1 = e1 / np.linalg.norm(e1, axis=1)[:,None]
        
    T2 =  np.cross(T1, np.cross(T1, e2))
    T2 /= np.linalg.norm(T2, axis=1)[:,None]
  
    # Arrays for the entries of the matrix   For L &
    data = []
    ii = []
    jj = []
    sqrtl = sqrt(llambda)
    b = np.zeros(3*F.shape[0], dtype=np.complex128)
    index = 0
    for f in range(F.shape[0]):
        for ei in range(3): # Loop over the edges
            
            # Look up the opposite face
            g = TT[f, ei]
            
            # If it is a boundary edge, it does not contribute to the energy
            # or avoid to count every edge twice f>g
            if g == -1 or f > g:
                continue
                
            # Compute the complex representation of the common edge
            e  = V[F[f, (ei+1)%3], :] - V[F[f, ei], :]
            
            vef = np.array([np.dot(e, T1[f, :]), np.dot(e, T2[f, :])])
            vef /= np.linalg.norm(vef)
            ef = (vef[0] + vef[1]*1j).conjugate()
            
            veg = np.array([np.dot(e, T1[g, :]), np.dot(e, T2[g, :])])
            veg /= np.linalg.norm(veg)
            eg = (veg[0] + veg[1]*1j).conjugate()
            
            if f in soft_id and g in soft_id:
                continue
            elif f in soft_id:
                fs = f
                i_vs = np.argwhere(soft_id==f) 
                vs = soft_value[i_vs, :]
                vec = np.array([np.dot(vs, T1[fs, :]),np.dot(vs, T2[fs, :])])
                vec /= np.linalg.norm(vec)
                c = vec[0] + vec[1] * 1j
                data.append(eg); ii.append(index); jj.append(g)   
                b[index] = c*ef
                
            elif g in soft_id:
                fs = g
                i_vs = np.argwhere(soft_id==g) 
                vs = soft_value[i_vs, :]
                vec = np.array([np.dot(vs, T1[fs, :]), np.dot(vs, T2[fs, :])])
                vec /= np.linalg.norm(vec)
                c = vec[0] + vec[1] * 1j
                data.append(ef);  ii.append(index); jj.append(f)             
                b[index] =  c*eg
                
            else:
                data.append(ef);  ii.append(index); jj.append(f)
                data.append(-eg); ii.append(index); jj.append(g)

            index += 1
    
    del_index = []
    for m in range(index + soft_id.shape[0],3*F.shape[0]):
        del_index.append(m)
    b = np.delete(b,del_index)
    
    for ci in range(soft_id.shape[0]):
        f = soft_id[ci]
        v = soft_value[ci, :]
        
        # Project on the local frame
        c = np.dot(v, T1[f, :]) + np.dot(v, T2[f, :])*1j
        
        data.append(1); ii.append(index); jj.append(f)
        b[index] = c * 1
 
        
        index += 1
    
    assert(b.shape[0] == index)
    

    # Solve the linear system
    A = sp.coo_matrix((data, (ii, jj)), shape=(index, F.shape[0])).asformat("csr")
#     print((A.H).shape, b.shape)
    u = sp.linalg.spsolve(A.H @ A, A.H @ b) #Then we get u
    #in order to visualize u, we need to covert it back 
    R = T1 * u.real[:,None] + T2 * u.imag[:,None]
    return R

In [7]:
R_hard = align_field_hard(v, f, tt, cf, c, 1e6)
plot_mesh_field(v, f, R_hard, cf)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

# Part2: Reconstructing a scalar field from a vector field

In [8]:
#Determine the matrix K and vector b in the above minimization (by expanding the least-squares error expression).
# K = G^T*A*G
# b = - 2*G^T*A*Ut
# c = Ut^T * Ut
u = R_hard.flatten('F')
c = 0
for i in u:
    c += i * i
G = igl.grad(v,f)
# print(G.shape, A.shape)
dbl_area = igl.doublearea(v, f)
A = sp.diags(np.concatenate([dbl_area,dbl_area,dbl_area],axis=0))
s = sp.linalg.spsolve((G.T @ A @ G), (G.T @ A @ u))
gt = (G@s).reshape(3,f.shape[0]).T
poss = sum(np.linalg.norm(gt-R_hard,axis = 1))
print("Poisson reconstruction error: ", poss)

Poisson reconstruction error:  170.47500267160478


In [9]:
def plot_scalar(V, F, R):    
    # Scaling of the representative vectors
    avg = igl.avg_edge_length(V, F)/2

    #Plot from face barycenters
    B = igl.barycenter(V, F)

    p = mp.plot(V, F, c=s)
    p.add_lines(B, B + R * avg)  
    return p

In [10]:
plot_scalar(v, f, gt-R_hard)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

## Part 3: Harmonic and LSCM Parameterizations

In [11]:
def plot_Harmonic(V, F, R):
    # Highlight in red the constrained faces 
    # Scaling of the representative vectors
    avg = igl.avg_edge_length(V, F)/2

    #Plot from face barycenters
    B = igl.barycenter(V, F)

    p = mp.plot(V, F)
    p.add_lines(B, B + R * avg)  
    return p

In [12]:
def align_field_camel(V, F, TT):

    
    # Edges
    e1 = V[F[:, 1], :] - V[F[:, 0], :]
    e2 = V[F[:, 2], :] - V[F[:, 0], :]

    # Compute the local reference systems for each face, T1, T2
    T1 = e1 / np.linalg.norm(e1, axis=1)[:,None]
        
    T2 =  np.cross(T1, np.cross(T1, e2))
    T2 /= np.linalg.norm(T2, axis=1)[:,None]
  
    # Arrays for the entries of the matrix   For L &
    data = []
    ii = []
    jj = []
    
    index = 0
    for f in range(F.shape[0]):
        for ei in range(3): # Loop over the edges
            
            # Look up the opposite face
            g = TT[f, ei]
            
            # If it is a boundary edge, it does not contribute to the energy
            # or avoid to count every edge twice f>g
            if g == -1 or f > g:
                continue
                
            # Compute the complex representation of the common edge
            e  = V[F[f, (ei+1)%3], :] - V[F[f, ei], :]
            
            vef = np.array([np.dot(e, T1[f, :]), np.dot(e, T2[f, :])])
            vef /= np.linalg.norm(vef)
            ef = (vef[0] + vef[1]*1j).conjugate()
            
            veg = np.array([np.dot(e, T1[g, :]), np.dot(e, T2[g, :])])
            veg /= np.linalg.norm(veg)
            eg = (veg[0] + veg[1]*1j).conjugate()
            
            # Add the term conj(f)^n*ui - conj(g)^n*uj to the energy matrix
            # The first one is used to store conjugate ef
            # The second one stores the row where ef is in the L
            # The third one stores the uf
            data.append(ef);  ii.append(index); jj.append(f)
            # similar, the minus because there's minus in dot before
            data.append(-eg); ii.append(index); jj.append(g)

            index += 1
            

    
    # Convert the constraints into the complex polynomial coefficients and add them as soft constraints
    # Attention!!! This is for the soft constraints
    # Rhs of the system
    b = np.ones(index, dtype=complex)
    
    assert(b.shape[0] == index)
 
    

    
    
    # Solve the linear system
    A = sp.coo_matrix((data, (ii, jj)), shape=(index, F.shape[0])).asformat("csr")
    u = sp.linalg.spsolve(A.H @ A, A.H @ b) #Then we get u
    #in order to visualize u, we need to covert it back 
    R = T1 * u.real[:,None] + T2 * u.imag[:,None]

    return R

In [13]:
# harmonic parametrization
v, f  = igl.read_triangle_mesh("data/camel_head.off")
bnd = igl.boundary_loop(f)
tt, _ = igl.triangle_triangle_adjacency(f)
R_camel = align_field_camel(v, f, tt)
u_camel = R_camel.flatten('F')
G = igl.grad(v,f)@v
dbl_area = igl.doublearea(v, f)
A = sp.diags(np.concatenate([dbl_area,dbl_area,dbl_area],axis=0))
s = sp.linalg.spsolve((G.T @ A @ G), (G.T @ A @ u_camel))
gt = (G@s).reshape(3,f.shape[0]).T
## Map the boundary to a circle, preserving edge proportions
bnd_uv = igl.map_vertices_to_circle(v, bnd)
## Harmonic parametrization for the internal vertices
uv = igl.harmonic_weights(v, f, bnd, bnd_uv, 1)
v_p = np.hstack([uv, np.zeros((uv.shape[0],1))])
p = mp.subplot(v, f, uv=uv, s=[1, 2, 0])
plot_Harmonic(v,f,gt)
mp.subplot(v_p, f,  shading={"wireframe": True}, data=p, s=[1, 2, 1])

/Users/yinuochang/miniforge3/envs/gm2022/lib/python3.9/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:144: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.9967555…

In [17]:
v, f = igl.read_triangle_mesh("data/camel_head.off")

# Fix two points on the boundary
b = np.array([2, 1])

bnd = igl.boundary_loop(f)
b[0] = bnd[0]
b[1] = bnd[int(bnd.size / 2)]

bc = np.array([[0.0, 0.0], [1.0, 0.0]])

# LSCM parametrization
_, uv = igl.lscm(v, f, b, bc)

p = mp.subplot(v, f, uv=uv, shading={"wireframe": False, "flat": False}, s=[1, 2, 0])
mp.subplot(uv, f, shading={"wireframe": True, "flat": False}, s=[1, 2, 1], data=p)
# plot(uv, F, shading={"wireframe": True}, plot=p3)

# @interact(mode=['3D','2D'])
# def switch(mode):
#     if mode == "3D":
#         plot(v, f, uv=uv, shading={"wireframe": False, "flat": False}, plot=p)
#     if mode == "2D":
#         plot(uv, f, uv=uv, shading={"wireframe": True, "flat": False}, plot=p)

# Editing the parameterization

In [15]:
v, f  = igl.read_triangle_mesh("data/irr4-cyl2.off")
bnd = igl.boundary_loop(f)
bnd_uv = igl.map_vertices_to_circle(v, bnd)
## Harmonic parametrization for the internal vertices
uv = igl.harmonic_weights(v, f, bnd, bnd_uv, 1)
v_p = np.hstack([uv, np.zeros((uv.shape[0],1))])
plot_scalar(v_p,f,R_hard)
plot_mesh_field(v_p, f, R_hard, cf)
p = mp.subplot(uv, f, c=v[:, 0], s=[1, 2, 0])
mp.subplot(uv, f, c=v[:, 0], shading={"wireframe": True, "flat": False}, data=p, s=[1, 2, 1])

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [32]:
v, f = igl.read_triangle_mesh("data/irr4-cyl2.off")
# Fix two points on the boundary
b = np.array([2, 1])

bnd = igl.boundary_loop(f)
b[0] = bnd[0]
b[1] = bnd[int(bnd.size / 2)]

bc = np.array([[0.0, 0.0], [1.0, 0.0]])

# LSCM parametrization
_, uv = igl.lscm(v, f, b, bc)

p = mp.subplot(v, f, uv=uv, shading={"wireframe": False, "flat": False}, s=[1, 2, 0])
mp.subplot(uv, f, shading={"wireframe": True, "flat": False}, s=[1, 2, 1], data=p)